In [181]:
import pulp

In [182]:
model_4_a = pulp.LpProblem("Multinamtional_company", pulp.LpMinimize)
Production_center = ["A", "B", "C", "D"]
Warehouse = [1, 2, 3]
Distribution_center = [1, 2, 3, 4, 5, 6]
X_1 = pulp.LpVariable.dicts(f"ProductionCenter_to_Warehouse", [(i,j) for i in Production_center for j in Warehouse], lowBound=0, cat=pulp.LpInteger)
X_2 = pulp.LpVariable.dicts(f"Warehouse_to_DistributionCenter", [(i,j) for i in Warehouse for j in Distribution_center], lowBound=0, cat=pulp.LpInteger)
Costs_fromProdctionToWarehouse_matrix=[
    [4, 3, 5],
    [6, 2, 4],
    [5, 4, 3],
    [7, 5, 6]
]
fist_half_route = [
    [X_1["A", i] for i in Warehouse],
    [X_1["B", i] for i in Warehouse],
    [X_1["C", i] for i in Warehouse],
    [X_1["D", i] for i in Warehouse],
]
Costs_fromProdctionToWarehouse = pulp.lpDot(Costs_fromProdctionToWarehouse_matrix, fist_half_route)
Costs_fromWarehouseToDistributionCenter_matrix=[
    [3, 2, 5, 4, 6, 7],
    [1, 4, 2, 3, 5, 6],
    [5, 3, 4, 2, 3, 4]
]
second_half_route = [
    [X_2[1, i] for i in Distribution_center],
    [X_2[2, i] for i in Distribution_center],
    [X_2[3, i] for i in Distribution_center],
]
Costs_fromWarehouseToDistributionCenter = pulp.lpDot(Costs_fromWarehouseToDistributionCenter_matrix, second_half_route)
Costs_fromWarehouseToDistributionCenter

model_4_a += Costs_fromProdctionToWarehouse + Costs_fromWarehouseToDistributionCenter, "Objective"

# flow in and out constriants
for i in Warehouse:
    model_4_a += pulp.lpSum(X_1[in_, i] for in_ in Production_center) == pulp.lpSum(X_2[i, out_] for out_ in Distribution_center), f"Equal_in_and_out_flow_ofWarehouse{i}"

# supply and demand cap
Prodiction_supply = [70, 90, 80, 60]
for index_ in range(len(Production_center)):
    model_4_a += pulp.lpSum(X_1[Production_center[index_], j] for j in Warehouse) == Prodiction_supply[index_], f"Supply_from_ProductionCenter{Production_center[index_]}"

Distribution_demand = [50, 40, 60, 70, 50, 30]
for index_ in range(len(Distribution_center)):
    model_4_a += pulp.lpSum(X_2[j, Distribution_center[index_]] for j in Warehouse) == Distribution_demand[index_], f"Demand_from_distributionCenter{Distribution_center[index_]}"
model_4_a.solve()
print("Model Status:{}".format(pulp.LpStatus[model_4_a.status]))
for v in model_4_a.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_4_a.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/9d26bfececa243f98c42fe4dbb13f15f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/9d26bfececa243f98c42fe4dbb13f15f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 169 RHS
At line 183 BOUNDS
At line 214 ENDATA
Problem MODEL has 13 rows, 30 columns and 60 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1700 - 0.00 seconds
Cgl0004I processed model has 13 rows, 30 columns (30 integer (0 of which binary)) and 60 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 1700 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc

In [183]:
model_4_a

Multinamtional_company:
MINIMIZE
4*ProductionCenter_to_Warehouse_('A',_1) + 3*ProductionCenter_to_Warehouse_('A',_2) + 5*ProductionCenter_to_Warehouse_('A',_3) + 6*ProductionCenter_to_Warehouse_('B',_1) + 2*ProductionCenter_to_Warehouse_('B',_2) + 4*ProductionCenter_to_Warehouse_('B',_3) + 5*ProductionCenter_to_Warehouse_('C',_1) + 4*ProductionCenter_to_Warehouse_('C',_2) + 3*ProductionCenter_to_Warehouse_('C',_3) + 7*ProductionCenter_to_Warehouse_('D',_1) + 5*ProductionCenter_to_Warehouse_('D',_2) + 6*ProductionCenter_to_Warehouse_('D',_3) + 3*Warehouse_to_DistributionCenter_(1,_1) + 2*Warehouse_to_DistributionCenter_(1,_2) + 5*Warehouse_to_DistributionCenter_(1,_3) + 4*Warehouse_to_DistributionCenter_(1,_4) + 6*Warehouse_to_DistributionCenter_(1,_5) + 7*Warehouse_to_DistributionCenter_(1,_6) + 1*Warehouse_to_DistributionCenter_(2,_1) + 4*Warehouse_to_DistributionCenter_(2,_2) + 2*Warehouse_to_DistributionCenter_(2,_3) + 3*Warehouse_to_DistributionCenter_(2,_4) + 5*Warehouse_to_Distri

In [ ]:
import copy
model_4_a.solve()
# record the default optimal solution
current_solution = {(t, s): X_2[t, s].varValue for t in Warehouse for s in Distribution_center if X_2[t, s].varValue > 0}
optimal_value = pulp.value(model_4_a.objective)

solutions = []
value_list = []
# record the first half route not used in the default optimal solution
all_combinations = [(t, s) for t in Warehouse for s in Distribution_center]
not_in_solution = [combo for combo in all_combinations if combo not in current_solution]

for route_ in not_in_solution:
    model_toSolve = copy.deepcopy(model_4_a)
    print(model_toSolve)
    # Force the constraint another optimal solution
    model_toSolve += X_2[route_] >= 5, f"Forced_contraint_on_Zero{route_}"
    model_toSolve.solve()
    value_list.append(pulp.value(model_toSolve.objective))
    if pulp.value(model_toSolve.objective) == optimal_value:
        current_solution = {(t, s): X_1[t, s].varValue for t in Production_center for s in Warehouse if X_1[t, s].varValue > 0}
        current_solution.update({(t, s): X_2[t, s].varValue for t in Warehouse for s in Distribution_center if X_2[t, s].varValue > 0})
        solutions.append(current_solution)
len(solutions)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/40818a6aefa14f17afc8f15169ff58c0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/40818a6aefa14f17afc8f15169ff58c0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 169 RHS
At line 183 BOUNDS
At line 214 ENDATA
Problem MODEL has 13 rows, 30 columns and 60 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1700 - 0.00 seconds
Cgl0004I processed model has 13 rows, 30 columns (30 integer (0 of which binary)) and 60 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 1700 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc

2

In [185]:
solutions

[{('A', 1): 40.0,
  ('A', 2): 30.0,
  ('B', 2): 90.0,
  ('C', 3): 80.0,
  ('D', 2): 60.0,
  (1, 1): 5.0,
  (1, 2): 40.0,
  (2, 1): 50.0,
  (2, 3): 60.0,
  (2, 4): 70.0,
  (3, 5): 50.0,
  (3, 6): 30.0},
 {('A', 1): 40.0,
  ('A', 2): 30.0,
  ('B', 2): 90.0,
  ('C', 3): 80.0,
  ('D', 2): 60.0,
  (1, 1): 5.0,
  (1, 2): 40.0,
  (1, 3): 5.0,
  (1, 4): 5.0,
  (1, 5): 5.0,
  (1, 6): 5.0,
  (2, 1): 50.0,
  (2, 2): 5.0,
  (2, 3): 60.0,
  (2, 4): 70.0,
  (2, 5): 5.0,
  (2, 6): 5.0,
  (3, 1): 5.0,
  (3, 5): 50.0,
  (3, 6): 30.0}]